In [1]:
from requests import get
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np

from numpy import array
from numpy import argmax
import keras
from keras.utils import to_categorical

Using TensorFlow backend.


Following code gets data for a selected number of movies released between 1934-01-01 and 2017-12-31, ordered by popularity (ordered by IMDB)

In [37]:
actorAwards = ['Actor', 'Actor in a Supporting Role', 'Actress', 'Actress in a Supporting Role', 'Actor in a Leading Role', 
               'Actress in a Leading Role']
#Awards that can be received directly by an actor/actress

directorAwards = ['Directing'] #For this dataset that name of the director is in the 'Film' category
#Awards that can be received directly by a director

movieAwards = ['Best Motion Picture', 'Outstanding Motion Picture', 'Documentary', 'Documentary (Feature)', 
               'Documentary (Short Subject)', 'Short Subject (Cartoon)', 'Short Subject (One Reel)',
               'Short Film (Animated)', 'Short Film (Live Action)', 'Best Picture', 'Animated Feature Film'] #Movie name is in the 'name' category
#Awards that directors and actors can directly influence

#Awards like writing, screenplay, production, etc are saved under other awards as the actor and director do not
#influence them as directly (or at all) as the ones above

OscarRead = pd.read_csv('Oscardatabase.csv') #read the database
yearCSV = OscarRead["Year"] #The year the movie was released
awardCSV = OscarRead["Award"] #The name of the award received/nominated for
winnerCSV = OscarRead["Winner"] #If the entry won or not

resultsCSV = winnerCSV.copy() #Create a copy to edit
resultsCSV[resultsCSV != 1] = 0 #Original CSV has a '1' for winner, and NaN for a nomination - convert NaN to '0'

nameCSV = OscarRead["Name"] 
filmCSV = OscarRead["Film"]

win = 1
nomination = 0

def getDict(awards, result = win, name = nameCSV, film = filmCSV, starAwards = True, otherAwards = False):
    """awards = list containing awards interested in.
       nameOrFilm = column that contains the actor/director name (different for different awards)
       Returns a new nested dictionary where the outside key is 
       a year, and the inside key is the name of actor/film/director
       who received a nomination or won an award in the list
       for the years 1935-2016"""
    
    Dict = {}
    for i in range(len(yearCSV)):
        if otherAwards == False and awardCSV[i] not in awards or resultsCSV[i] != result:
            continue
        elif otherAwards == True and awardCSV[i] in awards or resultsCSV[i] != result:
            continue
        if starAwards == False:
            try:
                try:
                    Dict[yearCSV[i] + 1][name[i]] += 1 #If the movie wins multiple awards in a year
                except:
                    Dict[yearCSV[i] + 1][name[i]] = 1 #Set the value for movie to one if the year is already in the dictionary
            except:
                Dict[yearCSV[i] + 1] = {} #if year is not in dictionary yet then create it
                Dict[yearCSV[i] + 1][name[i]] = 1 #set the value to one
                #Above code uses year + 1 for the key as the original database has the release year of the movie stored, 
                #not the year of the oscars
        else:
            try:
                try:
                    Dict[yearCSV[i] + 1][name[i]][1] += 1
                except:
                    Dict[yearCSV[i] + 1][name[i]] = film[i], 1
            except:
                Dict[yearCSV[i] + 1] = {}
                Dict[yearCSV[i] + 1][name[i]] = film[i], 1            
    return Dict
#Following dictionaries have data for 1935-2016
#If a movie/actor/director wins an award, it is only saved as a winner and will not have an entry for the nomination
#it received for that award
OscarActorWinners = getDict(actorAwards)
OscarActorNoms = getDict(actorAwards, nomination)
OscarDirWinners = getDict(directorAwards, name = filmCSV, film = nameCSV)
OscarDirNoms = getDict(directorAwards, nomination, name = filmCSV, film = nameCSV)
movieWinners = getDict(movieAwards, starAwards = False)
movieNoms = getDict(movieAwards, nomination, starAwards = False)
otherAwardWinners = getDict(actorAwards + movieAwards + directorAwards, starAwards = False, otherAwards = True)
otherAwardNoms = getDict(actorAwards + movieAwards + directorAwards, nomination, starAwards = False, otherAwards = True)

"""I tried adding data for 2017 and 2018 to the CSV above but it broke the code. The following code simply
reads from a new CSV containing this data and adds to the dictionaries above to make the total range 1935-2018"""

OscarRead2 = pd.read_csv('OscarDataContd.csv', encoding = 'utf-8')
yearCSV2 = OscarRead2["Year"] 
awardCSV2 = OscarRead2["Award"]
winnerCSV2 = OscarRead2["Winner"]

resultsCSV2 = winnerCSV2.copy()
resultsCSV2[resultsCSV2 != 1] = 0

nameCSV2 = OscarRead2["Name"]
filmCSV2 = OscarRead2["Film"]

def updateDict(awards, Dict, result = win, name = nameCSV2, film = filmCSV2, starAwards = True, otherAwards = False):
    """Essentially the same as getDict above however this updates a dictionary
       that is passed to the function"""
    
    for i in range(len(yearCSV2)):
        if otherAwards == False and awardCSV2[i] not in awards or resultsCSV2[i] != result:
            continue
        elif otherAwards == True and awardCSV2[i] in awards or resultsCSV2[i] != result:
            continue
        if starAwards == False:
            try:
                try:
                    Dict[yearCSV2[i] + 1][name[i]] += 1 #If the actor/director/movie wins multiple awards in a year
                except:
                    Dict[yearCSV2[i] + 1][name[i]] = 1 #Set the value for actor to one if the year is already in the dictionary
            except:
                Dict[yearCSV2[i] + 1] = {} #if year is not in dictionary yet then create it
                Dict[yearCSV2[i] + 1][name[i]] = 1 #set the value to one
        else:
            try:
                try:
                    Dict[yearCSV2[i] + 1][name[i]][1] += 1
                except:
                    Dict[yearCSV2[i] + 1][name[i]] = film[i], 1
            except:
                Dict[yearCSV2[i] + 1] = {}
                Dict[yearCSV2[i] + 1][name[i]] = film[i], 1   

#Update dictionaries to contain data for 1935-2018
updateDict(actorAwards, OscarActorWinners)
updateDict(actorAwards, OscarActorNoms, nomination)
updateDict(directorAwards, OscarDirWinners, name = filmCSV2, film = nameCSV2)
updateDict(directorAwards, OscarDirNoms, nomination, name = filmCSV2, film = nameCSV2)
updateDict(movieAwards, movieWinners, starAwards = False)
updateDict(movieAwards, movieNoms, nomination, starAwards = False)
updateDict(actorAwards + movieAwards + directorAwards, otherAwardWinners, starAwards = False, otherAwards = True)
updateDict(actorAwards + movieAwards + directorAwards, otherAwardNoms, nomination, starAwards = False, otherAwards = True)

print("1950 oscar winners (acting) are:", OscarActorWinners[1950])
print("1950 oscar nominees (acting) are:", OscarActorNoms[1950])
print("1950 oscar winners (directing) are:", OscarDirWinners[1950])
print("1950 oscar nominees (directing) are:", OscarDirNoms[1950])
print("1950 oscar winners (movie) are:", movieWinners[1950])
print("1950 oscar nominees (movie) are:", movieNoms[1950])
print("1950 oscar winners (other) are:", otherAwardWinners[1950])
print("1950 oscar nominees (other) are:", otherAwardNoms[1950])

otherAwardWinners[2018]

1950 oscar winners (acting) are: {'Broderick Crawford': ("All the King's Men", 1), 'Dean Jagger': ("Twelve O'Clock High", 1), 'Olivia de Havilland': ('The Heiress', 1), 'Mercedes McCambridge': ("All the King's Men", 1)}
1950 oscar nominees (acting) are: {'Kirk Douglas': ('Champion', 1), 'Gregory Peck': ("Twelve O'Clock High", 1), 'Richard Todd': ('The Hasty Heart', 1), 'John Wayne': ('Sands of Iwo Jima', 1), 'John Ireland': ("All the King's Men", 1), 'Arthur Kennedy': ('Champion', 1), 'Ralph Richardson': ('The Heiress', 1), 'James Whitmore': ('Battleground', 1), 'Jeanne Crain': ('Pinky', 1), 'Susan Hayward': ('My Foolish Heart', 1), 'Deborah Kerr': ('Edward, My Son', 1), 'Loretta Young': ('Come to the Stable', 1), 'Ethel Barrymore': ('Pinky', 1), 'Celeste Holm': ('Come to the Stable', 1), 'Elsa Lanchester': ('Come to the Stable', 1), 'Ethel Waters': ('Pinky', 1)}
1950 oscar winners (directing) are: {'Joseph L. Mankiewicz': ('A Letter to Three Wives ', 1)}
1950 oscar nominees (directing

{'Coco': 1,
 'The Shape of Water': 2,
 'Blade Runner 2049': 2,
 'Get Out': 1,
 'Call Me by Your Name': 1,
 'The Silent Child': 1,
 'Dunkirk': 3,
 'Dear Basketball': 1,
 'A Fantastic Woman': 1,
 'Phantom Thread': 1,
 'Darkest Hour': 1}

In [41]:
OscarActorWinners[2018]['Gary Oldman'][0] #Gary won an oscar for his role in Darkest Hour

'Darkest Hour'

In [42]:
OscarActorWinnersDupe = getDict(actorAwards, starAwards = False)
OscarActorNomsDupe = getDict(actorAwards, nomination, starAwards = False)
OscarDirWinnersDupe = getDict(directorAwards, name = filmCSV, film = nameCSV, starAwards = False)
OscarDirNomsDupe = getDict(directorAwards, nomination, name = filmCSV, film = nameCSV, starAwards = False)
updateDict(actorAwards, OscarActorWinnersDupe, starAwards = False)
updateDict(actorAwards, OscarActorNomsDupe, nomination, starAwards = False)
updateDict(directorAwards, OscarDirWinnersDupe, name = filmCSV2, film = nameCSV2, starAwards = False)
updateDict(directorAwards, OscarDirNomsDupe, nomination, name = filmCSV2, film = nameCSV2, starAwards = False)
#These are the same as the dictionaries above except that for the starAwards = False tag. Using this simply returns
#a dictionary of the form Dict[year][actor] = oscars won in year, instead of Dict[year][actor] = movie, num oscars
#Simply makes it easier to deal with just the numbers


startYear = 1934 + 1
endYear = 2018
def getRunningTotal(Dict):
    """Returns a dictionary that contains the amount
    of awards a actor/director has received at a given time"""
    totalDict = {}
    totalDict[startYear] = Dict[startYear]
            
    for year in range(startYear + 1, endYear + 1):
        totalDict[year] = {}
        totalDict[year].update(totalDict[year - 1]) #update dictionary with previous year to keep a running total each year
    
        for key, value in Dict[year].items():
            try:
                totalDict[year][key] += 1 #if the actor is already present increment by 1
            except:
                totalDict[year][key] = 1 #set it to 1 if not already present
    return totalDict

OscarActorWinnersTot = getRunningTotal(OscarActorWinnersDupe)
OscarActorNomsTot = getRunningTotal(OscarActorNomsDupe)
OscarDirWinnersTot = getRunningTotal(OscarDirWinnersDupe)
OscarDirNomsTot = getRunningTotal(OscarDirNomsDupe)

print(OscarActorWinnersTot[1953]['Anthony Quinn'])
print(OscarActorWinnersTot[1957]['Anthony Quinn']) #Anthony won an Oscar in 1953 and 1957

1
2


In [44]:
for key, value in OscarDirWinnersTot[2018].items(): #Directors that won multiple awards (just for fun lol)
    if value >= 2:
        print(key)
        print(value)
        
for key, value in OscarActorWinnersTot[2018].items(): #Actors that won multiple awards (just for fun lol)
    if value >= 3:
        print(key)
        print(value)

Frank Capra
3
John Ford
4
Leo McCarey
2
William Wyler
3
Billy Wilder
2
Elia Kazan
2
Joseph L. Mankiewicz
2
George Stevens
2
Fred Zinnemann
2
David Lean
2
Milos Forman
2
Oliver Stone
2
Clint Eastwood
2
Steven Spielberg
2
Ang Lee
2
Alejandro G. Iñárritu
2
Walter Brennan
3
Ingrid Bergman
3
Katharine Hepburn
3
Jack Nicholson
3
Meryl Streep
3


In [64]:
urlArray = []

numMovies = 1000 #numMovies must be less than 300,908, numMovies % 50 must be 0

for i in range(1, numMovies - 48, 50): #create an array of urls to iterate through
    urlArray.append('https://www.imdb.com/search/title?title_type=feature&release_date=1950-01-01,2017-12-31&start='
                    + str(i) + '&ref_=adv_nxt')

In [46]:
def cleanData(year_, genre, runtime, box_office):
    year1 = ''.join(c for c in year_ if c.isdigit()) #remove non-numeric characters from year
    
    genre1 = ''.join(c for c in genre if (c != '\n')) #original data has '\n' scattered around... remove those
    
    runtime1 = ''.join(c for c in runtime if c.isdigit()) #remove non-numeric characters from runtime
    
    box = ''.join(c for c in box_office if (c.isdigit() or c == '.')) #remove non-numeric characters, but keep the decimal
    box = round(float(box) * (10**6)) #original is in millions
    
    return int(year1), genre1, int(runtime1), box

In [47]:
def normalize(runtime, gross):
    nor_runtime = np.divide(runtime,max(runtime))
    nor_gross = np.divide(gross,max(gross))
    return nor_runtime, nor_gross

In [48]:
CPIdict = {}
CPIread = pd.read_csv('CPI.csv') #CPI.csv contains the multiplication factor to get from a given year to 2019 to adjust
                                 #for inflation
for i in range(len(CPIread["Growth"] + 1)):
     CPIdict[CPIread["Year"][i]] = CPIread["To 2019"][i]
print("Movies released in 1990 have a multiplier of", CPIdict[1990], "to their gross to adjust for inflation")

Movies released in 1990 have a multiplier of 1.9345070969999998 to their gross to adjust for inflation


In [49]:
def inflationAdjust(year, box_office):
    return int(round(box_office * CPIdict[year])) #multiplies the gross value by the multiplication factor for the year
                                                  #it was released

In [55]:
def getOscarNum(actor1, actor2, actor3, actor4, actor5, movieName, releaseYear):
    stars = []
    stars.extend([actor1, actor2, actor3, actor4, actor5])
    prevStarWins = prevStarNoms = starWins = starNoms = movieWins = movieNoms = otherWins = otherNoms = 0
    oscarYear = releaseYear + 1
    for star in stars:
        if star != '': #Checks how many oscar nominations and wins the directors and actors have prior to this movie
            prevStarWins += checkDict(star, OscarDirWinnersTot, releaseYear) #
            prevStarWins += checkDict(star, OscarActorWinnersTot, releaseYear)
            prevStarNoms += checkDict(star, OscarDirNomsTot, releaseYear)
            prevStarNoms += checkDict(star, OscarActorNomsTot, releaseYear)
            starWins += checkDict(star, OscarDirWinners, oscarYear, movie = movieName)
            starWins += checkDict(star, OscarActorWinners, oscarYear, movie = movieName)
            starNoms += checkDict(star, OscarDirNoms, oscarYear, movie = movieName)
            starNoms += checkDict(star, OscarActorNoms, oscarYear, movie = movieName)
    #Checks num of nominations/wins this movie received
    movieWins += checkDict(movieName, movieWinners, oscarYear)
    movieNoms += checkDict(movieName, movieNoms, oscarYear)
    otherWins += checkDict(movieName, otherAwardWinners, oscarYear)
    otherNoms += checkDict(movieName, otherAwardNoms, oscarYear)
    return prevStarWins, prevStarNoms, starWins, starNoms, movieWins, movieNoms, otherWins, otherNoms
        
def checkDict(name, Dict, year, movie = None):
    num = 0
    movieName = []
    if movie == None:
        try:
            num += Dict[year][name]       
        except:
            pass
    else:
        try:
            movieName = Dict[year][name][0]
            if movieName == movie:
                num += 1
        except:
            pass
    return num

In [56]:
x = getOscarNum('Gary Oldman', '', '', '', '', 'Darkest Hour', 2017)
x #Gary won an award for his role in Darkest Hour -- starWins is 1

(0, 1, 1, 0, 0, 0, 1, 3)

In [57]:
x = getOscarNum('Gary Oldman', '', '', '', '', 'not a real movie', 2017)
x #Gary didn't win an award for this movie -- starWins is 0

(0, 1, 0, 0, 0, 0, 0, 0)

In [65]:
names = []
year = []
genres = []
runtimes = []
metascores = []
imdbRatings = []
audienceRating = []
movieGross = []
directors =[]
movieGrossInflation = []
Star1 = []
Star2 = []
Star3 = []
Star4 = []
Star5 = []
res = []

missingData = 0

for url in urlArray:
    response = get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    movie_cont = html_soup.find_all('div', class_ = 'lister-item mode-advanced')
    
    for movie in movie_cont:
        name = movie.h3.a.text
        try:
            year_ = movie.h3.find('span', class_ = 'lister-item-year text-muted unbold').text
            genre = movie.p.find('span', class_ = 'genre').text
            runtime = movie.p.find('span', class_ = 'runtime').text
            viewer_rating = movie.p.span.text
            m_score = movie.find('span', class_ = 'metascore').text
            imdb_rating = float(movie.strong.text)
            box_office = movie.find_all('span', attrs = {'name': 'nv'})[1].text
            people = movie.find_all('p', class_ = '')[1]
            people = people.find_all('a')
            director = people[0].text
            actor1 = people[1].text
            
            #This try/except block is just in case lesser known movies are missing actors (might happen when
            #numMovies is set to like 10000, and I don't think it's a big deal as long as we have one actor + director)
            try:
                actor2 = people[2].text
            except:
                actor2 = ''
            try:
                actor3 = people[3].text
            except:
                actor3 = ''
            try:
                actor4 = people[4].text
            except:
                actor4 = ''
            try:
                actor5 = people[5].text
            except:
                actor5 =''
            
            year_, genre, runtime, box_office = cleanData(year_, genre, runtime, box_office) #clean up data
            
            box_office_inflation = inflationAdjust(year_, box_office) #adjust for inflation
            
            numStarWins, numStarNoms, starWin, starNom, numMovieWins, numMovieNoms, numOtherWins, numOtherNoms = getOscarNum(actor1, actor2, actor3, actor4, actor5, name, year_)
                
            res.append((name, year_, genre, runtime, director, actor1, actor2, actor3, actor4, actor5,
                        viewer_rating, m_score, imdb_rating, box_office, box_office_inflation, numStarWins, 
                        numStarNoms, starWin, starNom, numMovieWins, numMovieNoms, numOtherWins, numOtherNoms))
        except:
            print("Missing data for:", name)
            missingData += 1
            
(names, year, genres, runtimes, directors, Star1, Star2, Star3, Star4, Star5, audienceRatings, metascores, 
imdbRatings, movieGross, movieGrossInflation, starWinsList, starNomsList, starWon, starNom, movieWinsList, movieNomsList, 
otherWinsList, otherNomsList) = zip(*res)


runtime_normal, box_normal = normalize(runtimes, movieGrossInflation)#Normalize the number data

Missing data for: Love
Missing data for: I'm Not Here
Missing data for: Exposed
Missing data for: A Clockwork Orange
Missing data for: Unicorn Store
Missing data for: Contratiempo
Missing data for: McQ
Missing data for: Mine
Missing data for: Aterrados
Missing data for: Bright
Missing data for: Der Mann aus dem Eis
Missing data for: Gerald's Game
Missing data for: Caligola
Missing data for: Get Carter
Missing data for: Once Upon a Time in America
Missing data for: The Sand
Missing data for: Salò o le 120 giornate di Sodoma
Missing data for: It Came from Beneath the Sea
Missing data for: Flash Gordon
Missing data for: Elle
Missing data for: Brimstone
Missing data for: Così fan tutte
Missing data for: What Happened to Monday
Missing data for: The Room
Missing data for: The Autopsy of Jane Doe
Missing data for: Hawaii
Missing data for: The Promise
Missing data for: Srpski film
Missing data for: The Ritual
Missing data for: Habitación en Roma
Missing data for: Would You Rather
Missing data

In [59]:
print("Number of movies missing data:", missingData)
print("Total movies with data:", numMovies - missingData)

Number of movies missing data: 4
Total movies with data: 96


This part below takes in the genre list, it creates a one-hot list to make it numerical for regression analysis. I found that there are 191 different combinations of genres (for movies with multiple genres, it was found that the list of them is alphabetical).

In [60]:
genres_encoded = []
genres = [i.replace(" ","") for i in genres]
diff_genres = list(dict.fromkeys(genres))
char_to_int = dict((c, i) for i, c in enumerate(diff_genres))
int_to_char = dict((i, c) for i, c in enumerate(diff_genres))
print(len(char_to_int))
print(len(diff_genres))

integer_encoded = [char_to_int[char] for char in diff_genres]

encoded = list()
for value in integer_encoded:
    letter = [0 for _ in range(len(diff_genres))]
    letter[value] = 1
    encoded.append(letter)

for j in genres:
    encoded_value = char_to_int[j]
    genres_encoded.append(encoded[encoded_value])
print(len(genres_encoded))

49
49
96


In [61]:
df = pd.DataFrame({'Name': names,
                  'Year': year,
                  'Genre': genres,
                  'Genres One-Hot Encoded': genres_encoded,
                  'Director': directors,
                  'Star1' : Star1,
                  'Star2' : Star2,
                  'Star3' : Star3,
                  'Star4' : Star4,
                  'Star5' :Star5,
                  'Runtime': runtimes,
                  'Rating': audienceRatings,
                  'Metascore': metascores,
                  'IMDB rating': imdbRatings,
                  'Gross': movieGross,
                  'Inflation adjusted':movieGrossInflation,
                  'Runtime Normalized': runtime_normal,
                  'Box Office Adj. Norm.': box_normal,
                  'numStarW' : starWinsList,
                  'numStarN' : starNomsList,
                  'starWon' : starWon,
                  'starNom' :starNom,
                  'numMovieW' : movieWinsList,
                  'numMovieN' : movieNomsList,
                  'numOtherW' : otherWinsList,
                  'numOtherN' : otherNomsList})

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 26 columns):
Name                      96 non-null object
Year                      96 non-null int64
Genre                     96 non-null object
Genres One-Hot Encoded    96 non-null object
Director                  96 non-null object
Star1                     96 non-null object
Star2                     96 non-null object
Star3                     96 non-null object
Star4                     96 non-null object
Star5                     96 non-null object
Runtime                   96 non-null int64
Rating                    96 non-null object
Metascore                 96 non-null object
IMDB rating               96 non-null float64
Gross                     96 non-null int64
Inflation adjusted        96 non-null int64
Runtime Normalized        96 non-null float64
Box Office Adj. Norm.     96 non-null float64
numStarW                  96 non-null int64
numStarN                  96 non-null int64


In [63]:
df.head(numMovies)

,Name,Year,Genre,Genres One-Hot Encoded,Director,Star1,Star2,Star3,Star4,Star5,...,Runtime Normalized,Box Office Adj. Norm.,numStarW,numStarN,starWon,starNom,numMovieW,numMovieN,numOtherW,numOtherN
0,Room,2015,"Drama,Thriller","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Lenny Abrahamson,Brie Larson,Jacob Tremblay,Sean Bridgers,Wendy Crewson,,...,0.584158,0.015079,0,0,1,0,0,0,0,1
1,Guardians of the Galaxy,2014,"Action,Adventure,Comedy","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",James Gunn,Chris Pratt,Vin Diesel,Bradley Cooper,Zoe Saldana,,...,0.599010,0.342635,0,2,0,0,0,0,0,0
2,Get Out,2017,"Horror,Mystery,Thriller","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Jordan Peele,Daniel Kaluuya,Allison Williams,Bradley Whitford,Catherine Keener,,...,0.514851,0.174844,0,2,0,1,0,0,1,0
3,Scott Pilgrim vs. the World,2010,"Action,Comedy,Fantasy","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Edgar Wright,Michael Cera,Mary Elizabeth Winstead,Kieran Culkin,Anna Kendrick,,...,0.554455,0.035158,0,1,0,0,0,0,0,0
4,Thor: Ragnarok,2017,"Action,Adventure,Comedy","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Taika Waititi,Chris Hemsworth,Tom Hiddleston,Cate Blanchett,Mark Ruffalo,,...,0.643564,0.312919,2,7,0,0,0,0,0,0
5,The Greatest Showman,2017,"Biography,Drama,Musical","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Michael Gracey,Hugh Jackman,Michelle Williams,Zac Efron,Zendaya,,...,0.519802,0.173156,0,5,0,0,0,0,0,1
6,Mary Poppins,1964,"Comedy,Family,Fantasy","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Robert Stevenson,Julie Andrews,Dick Van Dyke,David Tomlinson,Glynis Johns,,...,0.688119,0.802732,0,1,1,0,0,0,0,0
7,Captain America: The First Avenger,2011,"Action,Adventure,Sci-Fi","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",Joe Johnston,Chris Evans,Hugo Weaving,Samuel L. Jackson,Hayley Atwell,,...,0.613861,0.191190,0,1,0,0,0,0,0,0
8,The Avengers,2012,"Action,Adventure,Sci-Fi","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",Joss Whedon,Robert Downey Jr.,Chris Evans,Scarlett Johansson,Jeremy Renner,,...,0.707921,0.660991,0,4,0,0,0,0,0,0
9,Kong: Skull Island,2017,"Action,Adventure,Fantasy","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...",Jordan Vogt-Roberts,Tom Hiddleston,Samuel L. Jackson,Brie Larson,John C. Reilly,,...,0.584158,0.166908,1,2,0,0,0,0,0,1
